In [1]:
import sys

sys.path.append("../../")

from app.common.logger import logger
from app.config.utils import Configuration, init_config
from app.common.llm_clients.openai_client import OpenAILLMClient

from ml.data_processing.prompt import STEP_1_3_QUESTION_GENERATION_PROMPT

In [2]:
config = Configuration()
init_config(config)
config = config()

# 데이터 불러오기

In [4]:
import json

In [ ]:
with open("../data/amr_guide_textbook_merged.json", "r", encoding="utf-8") as f:
    arm_guide_textbook = json.load(f)

In [23]:
with open("../data/acs_alarm_textbook_merged.json", "r", encoding="utf-8") as f:
    acs_alarm_textbook = json.load(f)

In [7]:
with open("../data/mcs_textbook_merged.json", "r", encoding="utf-8") as f:
    mcs_textbook = json.load(f)
with open("../data/hacs_textbook_merged.json", "r", encoding="utf-8") as f:
    hacs_textbook = json.load(f)
with open("../data/error_scenario_textbook_merged.json", "r", encoding="utf-8") as f:
    error_textbook = json.load(f)

In [9]:
len(mcs_textbook), len(hacs_textbook), len(error_textbook)

(20, 76, 4)

# 질문 생성

In [10]:
model = "google/gemini-3-flash-preview"
provider = "openrouter"
api_key = config["openrouter"]["api_key"]
params = {
    "max_tokens": 4096,
    "temperature": 0.0
}
llm_client = OpenAILLMClient(model=model, provider=provider, api_key=api_key, params=params)

2026-01-07 11:03:53.684 | INFO     | app.common.llm_clients.openai_client:_create_clients:57 - Creating OpenRouter client.


In [11]:
import re

def parse_json(text: str):
    """JSON 문자열을 파싱합니다."""
    match = re.search(r"```json\s*([\s\S]*?)\s*```", text)
    json_string = match.group(1).strip() if match else text
    return json.loads(json_string)

In [14]:
for x in arm_guide_textbook:
    try:
        questions = x['questions']
        logger.info('already have questions')
    except:
        textbook = x['textbook']
        user_prompt = f"""[Context]
    {textbook}"""
        try:
            response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
            result = response.choices[0].message.content
            paresed = parse_json(result)
            logger.info(paresed)
        except:
            logger.error(f"Error")
            try:
                response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
                result = response.choices[0].message.content
                paresed = parse_json(result)
                logger.info(paresed)
            except:
                logger.error(f"Error2")
                continue

        x['questions'] = paresed

2026-01-05 15:31:35.227 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.228 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.228 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.229 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.229 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.229 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.229 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.230 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.230 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.230 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.230 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31:35.231 | INFO     | __main__:<module>:4 - already have questions
2026-01-05 15:31

In [15]:
with open("amr_guide_textbook_question.json", "w", encoding="utf-8") as f:
    json.dump(arm_guide_textbook, f, ensure_ascii=False, indent=4)

In [24]:
for x2 in acs_alarm_textbook:
    try:
        questions = x2['questions']
        logger.info('already have questions')
    except:
        textbook = x2['textbook']
        user_prompt = f"""[Context]
{textbook}"""
        try:
            response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
            result = response.choices[0].message.content
            paresed = parse_json(result)
            logger.info(paresed)
        except:
            logger.error(f"Error")
            try:
                response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
                result = response.choices[0].message.content
                paresed = parse_json(result)
                logger.info(paresed)
            except:
                logger.error(f"Error2")
                continue

        x2['questions'] = paresed

2026-01-05 15:46:50.521 | INFO     | __main__:<module>:13 - ['BMA 자동화 시스템 내 AMR 자율주행 및 미션 제어의 논리적 아키텍처와 예외 처리 메커니즘 전반에 대해 기술 백서 형식으로 상세히 서술하시오.', 'AMR의 측위 보정을 위한 RFID 기반 절대 위치 인식 과정과 정밀 도킹 시 Lidar, 비전, IR 센서가 융합되어 목표 형상을 인지하고 정렬하는 작동 원리와 시스템적 상호작용을 공학적으로 분석하시오.', '추측 항법의 오차 임계점 도달, 액추에이터의 폐루프 제어 실패, 그리고 PLC와의 상태 불일치 상황 발생 시 시스템의 논리적 판단 과정과 대응 메커니즘을 서술하시오.']
2026-01-05 15:46:53.500 | INFO     | __main__:<module>:13 - ['BMA 자동화 시스템의 AMR 주행 제어 및 안전 메커니즘의 논리적 구조 전반에 대해 기술 백서 형식으로 상세히 서술하시오.', 'AMR의 자율 주행 시 전역 및 지역 경로 계획 알고리즘이 코스트맵(Costmap) 데이터와 결합하여 최적 궤적을 생성하는 메커니즘과, 센서 퓨전 데이터가 구동부의 키네마틱스 제어에 반영되는 시스템적 상호작용을 공학적으로 분석하시오.', '휠 슬립(Wheel Slip)이나 턴테이블 정렬 오차와 같은 물리적 임계 상황 발생 시, 시스템이 센서 간 데이터 불일치를 감지하고 안전 인터록을 활성화하는 논리적 판단 과정과 단계별 대응 메커니즘을 서술하시오.']
2026-01-05 15:46:56.612 | INFO     | __main__:<module>:13 - ['현대자동차그룹 BMA 공정 내 AMR(Autonomous Mobile Robot) 시스템 아키텍처의 전력 관리, 센서 퓨전, 구동 제어 및 네트워크 통신 전반에 대해 기술 백서 형식으로 상세히 서술하시오.', 'AMR의 공간 인지 무결성을 확보하기 위한 LiDAR, 카메라, IMU 등 이종 센서 간의 데이터 융합 메커니즘과, 각 센서의

In [25]:
with open("acs_alarm_textbook_question.json", "w", encoding="utf-8") as f:
    json.dump(acs_alarm_textbook, f, ensure_ascii=False, indent=4)

In [12]:
for x3 in mcs_textbook:
    try:
        questions = x3['questions']
        logger.info('already have questions')
    except:
        textbook = x3['textbook']
        user_prompt = f"""[Context]
{textbook}"""
        try:
            response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
            result = response.choices[0].message.content
            paresed = parse_json(result)
            logger.info(paresed)
        except:
            logger.error(f"Error")
            try:
                response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
                result = response.choices[0].message.content
                paresed = parse_json(result)
                logger.info(paresed)
            except:
                logger.error(f"Error2")
                continue

        x3['questions'] = paresed

with open("../data/mcs_textbook_question.json", "w", encoding="utf-8") as f:
    json.dump(mcs_textbook, f, ensure_ascii=False, indent=4)

2026-01-07 11:05:57.637 | INFO     | __main__:<module>:13 - ["BMA 공정 자동화 시스템 내 MCS와 PLC 간의 데이터 동기화 및 센서 제어를 위한 'PLC 센서 값 업데이트 API' 전반에 대해 기술 백서 형식으로 상세히 서술하시오.", 'API 요청 시 전달되는 SensorId와 Current 데이터가 실제 PLC 메모리 레지스터 주소로 매핑되어 물리적 장치에 기록되는 동작 메커니즘과 시스템적 상호작용을 공학적으로 분석하시오.', '센서 노후화에 따른 오차 발생이나 하드웨어 고장 상황에서 수동 오버라이드 및 보정 로직이 공정 중단을 방지하는 논리적 판단 과정과 대응 메커니즘을 서술하시오.']
2026-01-07 11:06:00.782 | INFO     | __main__:<module>:13 - ["MCS Backend 서버의 가용성 진단을 위한 'Hello' API의 정의, 명세 및 활용 시나리오 전반에 대해 기술 백서 형식으로 상세히 서술하시오.", '서버 연결 상태 확인 API가 시스템 초기화 및 헬스 체크 과정에서 클라이언트 및 로드 밸런서와 통신하는 작동 원리와 시스템적 상호작용을 공학적으로 분석하시오.', '특정 AMR의 명령 수신 불가 상황 발생 시, 인증 절차가 배제된 진단 API를 통해 네트워크 단절 여부를 격리(Isolation) 판단하는 논리적 과정과 대응 메커니즘을 서술하시오.']
2026-01-07 11:06:03.874 | INFO     | __main__:<module>:13 - ["MCS(Material Control System) 내 '전체 장소 상태 조회(Get All Places) API'의 명세와 운영 시나리오 전반에 대해 기술 백서 형식으로 상세히 서술하시오.", '해당 API가 반환하는 5가지 핵심 상태 데이터(Code, Alarm, Mode, Run, Logistics)가 실시간 관제 시스템 및 AMR 미션 할당 로직 내에서 어떻게 상호작용하는지 그 계통적 흐름을

In [13]:
for x4 in hacs_textbook:
    try:
        questions = x4['questions']
        logger.info('already have questions')
    except:
        textbook = x4['textbook']
        user_prompt = f"""[Context]
{textbook}"""
        try:
            response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
            result = response.choices[0].message.content
            paresed = parse_json(result)
            logger.info(paresed)
        except:
            logger.error(f"Error")
            try:
                response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
                result = response.choices[0].message.content
                paresed = parse_json(result)
                logger.info(paresed)
            except:
                logger.error(f"Error2")
                continue

        x4['questions'] = paresed

with open("../data/hacs_textbook_question.json", "w", encoding="utf-8") as f:
    json.dump(hacs_textbook, f, ensure_ascii=False, indent=4)

2026-01-07 11:07:09.246 | INFO     | __main__:<module>:13 - ['BMA 제어 시스템의 사용자 등록 API(/api/user/register) 명세와 운영 환경에서의 활용 방안에 대해 기술 백서 형식으로 상세히 서술하시오.', '사용자 식별 정보(UserId)와 인증 자격 증명(Password)이 데이터베이스에 적재되는 과정에서 보안 해시 알고리즘이 적용되는 원리와 시스템적 상호작용을 공학적으로 분석하시오.', 'UserId 중복 또는 데이터베이스 연결 오류와 같은 예외 상황 발생 시 시스템의 논리적 판단 과정과 응답 코드(500 Error)를 통한 대응 메커니즘을 서술하시오.']
2026-01-07 11:07:12.339 | INFO     | __main__:<module>:13 - ["ACS WebBackend의 '사용자 로그인 및 인증 토큰 발급(User Login & Token Issuance)' API 전반에 대해 기술 백서 형식으로 상세히 서술하시오.", 'JWT 기반 AuthToken 발급 과정에서 사용자 권한(RBAC) 및 클라이언트 식별자(ClientId)가 시스템 내 접근 제어 및 세션 관리와 상호작용하는 공학적 메커니즘을 분석하시오.', '시스템 라이선스 정책에 따른 최대 동시 접속자 수 초과(406 Not Acceptable) 상황 발생 시, 시스템의 논리적 판단 과정과 세션 점유 방지를 위한 대응 메커니즘을 서술하시오.']
2026-01-07 11:07:15.376 | INFO     | __main__:<module>:13 - ["BMA 공정 시스템의 보안 유지 및 세션 관리를 위한 '사용자 로그아웃(User Logout) API' 전반에 대해 기술 백서 형식으로 상세히 서술하시오.", '로그아웃 요청 시 전달되는 JWT(JSON Web Token)의 검증 과정과 서버 측 블랙리스트 등록을 통한 인증 무효화의 작동 원리와 시스템적 상호작용을 공학적으로 분석하시오.', '작업자 교대 및 시스

In [14]:
for x5 in error_textbook:
    try:
        questions = x5['questions']
        logger.info('already have questions')
    except:
        textbook = x5['textbook']
        user_prompt = f"""[Context]
{textbook}"""
        try:
            response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
            result = response.choices[0].message.content
            paresed = parse_json(result)
            logger.info(paresed)
        except:
            logger.error(f"Error")
            try:
                response = llm_client.generate(system_prompt=STEP_1_3_QUESTION_GENERATION_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
                result = response.choices[0].message.content
                paresed = parse_json(result)
                logger.info(paresed)
            except:
                logger.error(f"Error2")
                continue

        x5['questions'] = paresed

with open("../data/error_scenario_textbook_question.json", "w", encoding="utf-8") as f:
    json.dump(error_textbook, f, ensure_ascii=False, indent=4)

2026-01-07 11:11:01.197 | INFO     | __main__:<module>:13 - ['AMR의 도킹 전 위치 인식 실패 시 발생하는 에러 대응 및 복구 절차 전반에 대해 기술 백서 형식으로 상세히 서술하시오.', '상위 시스템(MCS)과 로봇 제어 시스템(ACS) 간의 상태 불일치를 해소하기 위한 API 호출 흐름과 각 단계별 데이터 필터링 및 시스템적 상호작용을 공학적으로 분석하시오.', 'DOCK_NOT_FIND 에러 발생 시 작업 취소부터 미션 재할당 및 AUTO 모드 전환까지 이어지는 논리적 판단 과정과 안전한 재시도를 위한 대응 메커니즘을 서술하시오.']
2026-01-07 11:11:03.985 | INFO     | __main__:<module>:13 - ['AMR 도킹 프로세스 중 발생하는 알람 시나리오와 이에 따른 표준 복구 절차(SOP) 전반에 대해 기술 백서 형식으로 상세히 서술하시오.', '도킹 오류 식별을 위한 API 상태 조회부터 MCS 미션 재할당 및 AUTO 모드 전환까지, 각 제어 단계가 전체 자동화 시스템 내에서 어떻게 상호작용하는지 공학적으로 분석하시오.', '도킹 위치 오차(80023) 또는 시간 초과(80024) 상황 발생 시, 2차 사고 방지를 위한 Job 취소 논리와 물리적 안전 확보를 위한 단계별 대응 메커니즘을 서술하시오.']
2026-01-07 11:11:06.681 | INFO     | __main__:<module>:13 - ['AMR 차상 작업 전 알람 발생 시의 표준 복구 절차(Scenario 3) 전반에 대해 기술 백서 형식으로 상세히 서술하시오.', '장애 발생 시 AMR의 상태 조회부터 PLC 인터페이스 초기화(dock_reset/ok), 그리고 MCS 미션 재시도까지 이어지는 제어 API 간의 시스템적 상호작용을 공학적으로 분석하시오.', '적재물 센서 미감지 및 신호 인터페이스 타임아웃 상황 발생 시, 시스템이 작업을 강제 취소하고 설비 상태를 재설정해야 하는 논리적 인

# 데이터 포맷팅 

In [15]:
system_content = "당신은 BMA(Battery Module Assembly) 공정의 자동화 시스템을 설계한 수석 아키텍트(Chief Architect)이자 공학 교수입니다."

In [16]:
think_templates = {
    0: """<think>
사용자가 '{topic}'에 대한 기술 백서 형식의 포괄적인 서술을 요청했다.
이에 따라 BMA 지식 베이스에서 해당 주제를 다루는 전체 챕터를 인출하여, 시스템 아키텍처와 엔지니어링 세부 사항을 상세히 설명한다.
</think>""",

    1: """<think>
사용자가 '{topic}'의 구체적인 공학적 작동 원리와 메커니즘에 대해 질문했다. ({question})
이 기술 요소는 단독으로 작동하지 않고 주변 시스템(PLC, 센서 등)과 유기적으로 연동되므로, 정확한 답변을 위해 전체 기술 맥락이 포함된 상세 백서를 제시하여 상호작용 흐름을 설명한다.
</think>""",

    2: """<think>
사용자가 '{topic}' 관련 상황에서의 논리적 인과관계와 시스템 대응 방식을 문의했다. ({question})
단순한 에러 해결책 제시보다 근본적인 발생 원인 분석과 Safety Interlock 로직을 이해하는 것이 중요하므로, 관련 프로토콜이 기술된 전문 분석 문서를 인출하여 답변한다.
</think>"""
}

In [28]:
result = []
for i, json_result in enumerate(arm_guide_textbook):
    if i < 7:
        text = json_result['preprocessed_group']
        textbook = json_result['textbook']
        chapter_title = text.split('\n')[0].strip().replace('#', '').strip()
        logger.info(chapter_title)
    else:
        textbook = json_result['textbook']
        chapter_title = textbook.split('\n')[0].strip().replace('#', '').strip()
        logger.info(chapter_title)

    questions = json_result['questions']
    for j, q in enumerate(questions):
        if j == 0:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title)
        else:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title, question=q)
        conversation = {
            "conversation": [
                {
                    "role": "system",
                    "content": system_content
                },
                {
                    "role": "user",
                    "content": q
                },
                {
                    "role": "assistant",
                    "content": think_content + '\n\n' + textbook
                }
            ]
        }
        result.append(conversation)

2026-01-05 15:51:32.545 | INFO     | __main__:<module>:7 - 1. Daily Operating
2026-01-05 15:51:32.546 | INFO     | __main__:<module>:7 - 2. Trouble Shooting: Call Assignment
2026-01-05 15:51:32.547 | INFO     | __main__:<module>:7 - 3. Trouble Shooting: Navigation & Obstacle
2026-01-05 15:51:32.547 | INFO     | __main__:<module>:7 - 4. Trouble Shooting: Docking & H/W
2026-01-05 15:51:32.547 | INFO     | __main__:<module>:7 - 5. Trouble Shooting: Model Specific
2026-01-05 15:51:32.548 | INFO     | __main__:<module>:7 - 6. Operational Manual: Control Tools (조작 및 유지보수 매뉴얼)
2026-01-05 15:51:32.548 | INFO     | __main__:<module>:7 - 7. Appendix
2026-01-05 15:51:32.548 | INFO     | __main__:<module>:11 - WEB ACS 기반의 Place Cycle 제어 논리와 미션 디스패칭(Mission Dispatching) 메커니즘
2026-01-05 15:51:32.549 | INFO     | __main__:<module>:11 - WEB ACS 기반의 동적 물류 제어 메커니즘: Place Cycle 및 가동 상태 동기화 프로토콜
2026-01-05 15:51:32.549 | INFO     | __main__:<module>:11 - BMA 공정 내 자율주행 로봇(AMR)의 전력 관리 및 운영 생명주기 프로토콜 (Energy

In [29]:
len(result)

192

In [30]:
with open("arm_guide_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [31]:
result2 = []
for i, json_result in enumerate(acs_alarm_textbook):
    textbook = json_result['textbook']
    chapter_title = textbook.split('\n')[0].strip().replace('#', '').strip()
    logger.info(chapter_title)

    questions = json_result['questions']
    for j, q in enumerate(questions):
        if j == 0:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title)
        else:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title, question=q)
        conversation = {
            "conversation": [
                {
                    "role": "system",
                    "content": system_content
                },
                {
                    "role": "user",
                    "content": q
                },
                {
                    "role": "assistant",
                    "content": think_content + '\n\n' + textbook
                }
            ]
        }
        result2.append(conversation)

2026-01-05 15:51:49.456 | INFO     | __main__:<module>:5 - BMA 자동화 시스템 기술 백서 (Technical Whitepaper)
2026-01-05 15:51:49.457 | INFO     | __main__:<module>:5 - BMA 자동화 시스템 기술 백서 (Technical Whitepaper)
2026-01-05 15:51:49.457 | INFO     | __main__:<module>:5 - 현대자동차그룹 BMA 공정 자동화 시스템 기술 백서
2026-01-05 15:51:49.457 | INFO     | __main__:<module>:5 - BMA 자동화 시스템 기술 백서: AMR 구동 제어 및 결함 허용 아키텍처
2026-01-05 15:51:49.458 | INFO     | __main__:<module>:5 - BMA 자동화 시스템 기술 백서 (Technical Whitepaper)
2026-01-05 15:51:49.458 | INFO     | __main__:<module>:5 - HMG BMA 자동화 시스템 기술 백서 (Technical Whitepaper)
2026-01-05 15:51:49.459 | INFO     | __main__:<module>:5 - BMA 자동화 시스템 기술 백서 (Technical Whitepaper)
2026-01-05 15:51:49.459 | INFO     | __main__:<module>:5 - BMA 자동화 시스템 기술 백서: AMR 미션 크리티컬 에러와 복구 메커니즘 분석
2026-01-05 15:51:49.459 | INFO     | __main__:<module>:5 - BMA 자동화 시스템 기술 백서: AMR 도킹 메커니즘 및 예외 처리 아키텍처
2026-01-05 15:51:49.459 | INFO     | __main__:<module>:5 - BMA 공정 내 AMR 도킹 정밀 제어 및 예외 처리 메커니즘 (Prec

In [32]:
len(result2)

99

In [33]:
with open("acs_alarm_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result2, f, ensure_ascii=False, indent=4)

In [17]:
result3 = []
for i, json_result in enumerate(mcs_textbook):
    textbook = json_result['textbook']
    chapter_title = textbook.split('\n')[0].strip().replace('#', '').strip()
    logger.info(chapter_title)

    questions = json_result['questions']
    for j, q in enumerate(questions):
        if j == 0:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title)
        else:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title, question=q)
        conversation = {
            "conversation": [
                {
                    "role": "system",
                    "content": system_content
                },
                {
                    "role": "user",
                    "content": q
                },
                {
                    "role": "assistant",
                    "content": think_content + '\n\n' + textbook
                }
            ]
        }
        result3.append(conversation)

logger.info(len(result3))

with open("../data/mcs_textbook_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result3, f, ensure_ascii=False, indent=4)

2026-01-07 11:11:18.766 | INFO     | __main__:<module>:5 - [API] PLC 센서 값 업데이트 (Update PLC Sensor Value)
2026-01-07 11:11:18.767 | INFO     | __main__:<module>:5 - [API] 서버 연결 상태 확인 (Server Connectivity Check)
2026-01-07 11:11:18.767 | INFO     | __main__:<module>:5 - [API] 전체 장소 상태 조회 (Get All Places)
2026-01-07 11:11:18.767 | INFO     | __main__:<module>:5 - [API] 장소 상태 정보 갱신 (Update Place Status)
2026-01-07 11:11:18.768 | INFO     | __main__:<module>:5 - [API] 전체 호출 정보 조회 (Retrieve All Calls)
2026-01-07 11:11:18.768 | INFO     | __main__:<module>:5 - [API] 전체 미션 목록 조회 (Retrieve All Missions)
2026-01-07 11:11:18.768 | INFO     | __main__:<module>:5 - [API] 호출 장소 상태 조회 (Retrieve Call Place Status)
2026-01-07 11:11:18.768 | INFO     | __main__:<module>:5 - [API] MCS 알람 정보 조회 (Retrieve MCS Alarms)
2026-01-07 11:11:18.769 | INFO     | __main__:<module>:5 - [API] AMR 이동 명령 (AMR Move Command)
2026-01-07 11:11:18.769 | INFO     | __main__:<module>:5 - [API] 호출 상태 변경 (Change Call State)
2026

In [18]:
result4 = []
for i, json_result in enumerate(hacs_textbook):
    textbook = json_result['textbook']
    chapter_title = textbook.split('\n')[0].strip().replace('#', '').strip()
    logger.info(chapter_title)

    questions = json_result['questions']
    for j, q in enumerate(questions):
        if j == 0:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title)
        else:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title, question=q)
        conversation = {
            "conversation": [
                {
                    "role": "system",
                    "content": system_content
                },
                {
                    "role": "user",
                    "content": q
                },
                {
                    "role": "assistant",
                    "content": think_content + '\n\n' + textbook
                }
            ]
        }
        result4.append(conversation)

logger.info(len(result4))
with open("../data/hacs_textbook_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result4, f, ensure_ascii=False, indent=4)

2026-01-07 11:11:25.414 | INFO     | __main__:<module>:5 - [API] 사용자 등록 (User Registration)
2026-01-07 11:11:25.414 | INFO     | __main__:<module>:5 - [API] 사용자 로그인 및 인증 토큰 발급 (User Login & Token Issuance)
2026-01-07 11:11:25.415 | INFO     | __main__:<module>:5 - [API] 사용자 로그아웃 (User Logout)
2026-01-07 11:11:25.415 | INFO     | __main__:<module>:5 - [API] 실행 중인 작업 오더 조회 (Get Running Work Orders)
2026-01-07 11:11:25.416 | INFO     | __main__:<module>:5 - [API] 작업 이력 조회 (Retrieve Work Order History)
2026-01-07 11:11:25.416 | INFO     | __main__:<module>:5 - [API] 작업 오더 삭제 (Delete Work Order)
2026-01-07 11:11:25.416 | INFO     | __main__:<module>:5 - [API] 객체 제어기 연결 테스트 (Object Controller Connectivity Test)
2026-01-07 11:11:25.417 | INFO     | __main__:<module>:5 - [API] 서비스 가용성 확인 (Hello Test Endpoint)
2026-01-07 11:11:25.417 | INFO     | __main__:<module>:5 - [API] 맵 객체 전체 조회 (Retrieve All Map Objects)
2026-01-07 11:11:25.417 | INFO     | __main__:<module>:5 - [API] 맵 스타일 구성 정보 조회 (Get

In [19]:
result5 = []
for i, json_result in enumerate(error_textbook):
    textbook = json_result['textbook']
    chapter_title = textbook.split('\n')[0].strip().replace('#', '').strip()
    logger.info(chapter_title)

    questions = json_result['questions']
    for j, q in enumerate(questions):
        if j == 0:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title)
        else:
            think_content = think_templates[j]
            think_content = think_content.format(topic=chapter_title, question=q)
        conversation = {
            "conversation": [
                {
                    "role": "system",
                    "content": system_content
                },
                {
                    "role": "user",
                    "content": q
                },
                {
                    "role": "assistant",
                    "content": think_content + '\n\n' + textbook
                }
            ]
        }
        result5.append(conversation)

logger.info(len(result5))
with open("../data/error_scenario_textbook_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result5, f, ensure_ascii=False, indent=4)

2026-01-07 11:11:28.254 | INFO     | __main__:<module>:5 - [SOP] 도킹 전 위치 인식 실패 시 복구 절차 (Scenario 1)
2026-01-07 11:11:28.254 | INFO     | __main__:<module>:5 - [SOP] 도킹 중 알람 발생 시 복구 절차 (Scenario 2)
2026-01-07 11:11:28.255 | INFO     | __main__:<module>:5 - [SOP] 차상 작업 전 알람 발생 시 복구 절차 (Scenario 3)
2026-01-07 11:11:28.256 | INFO     | __main__:<module>:5 - [SOP] 차상 작업 중 알람 발생 시 복구 절차 (Scenario 4)
2026-01-07 11:11:28.256 | INFO     | __main__:<module>:33 - 12
